In [1]:
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import os
import pandas as pd
import sqlalchemy as sa
import sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('main.ipynb'))))

import database as db
from models import County, DataTrustedIdentifier, TrustedIdentifier, DataSP, Image, create_data_trusted_identifier

## Database connection

In [3]:
engine, session = db.connect()
engine.echo = False

2023-03-24 16:32:12,497 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-03-24 16:32:12,498 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-24 16:32:12,499 INFO sqlalchemy.engine.Engine select current_schema()
2023-03-24 16:32:12,500 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-03-24 16:32:12,501 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-03-24 16:32:12,501 INFO sqlalchemy.engine.Engine [raw sql] {}


## List with full name and variations of identifier

In [4]:
list_identifier_trusted = {
    'full_name': ['Aline Vieira de Melo Silva', 'Carmen Lúcia Falcão Ichaso', 'Daniele Monteiro Ferreira', 'Daniel Ruschel', 'Elsie Franklin Guimarães', 'Eric J Tepe', 'Erika Erika Von Sohsten de Souza Medeiros', 'George Azevedo de Queiroz', 'Micheline Carvalho-Silva', 'Ricardo de la Merced Callejas Posada', 'Truman George Yuncker', 'William Trelease'],
    'searched_name': ['Silva', 'Ichaso', 'Monteiro', 'Ruschel', 'Guimar', 'Tepe', 'Medeiros', 'Queiroz', 'Carvalho', 'Callejas', 'Yuncker', 'Trelease']
}

pd.DataFrame(list_identifier_trusted)

,full_name,searched_name
0,Aline Vieira de Melo Silva,Silva
1,Carmen Lúcia Falcão Ichaso,Ichaso
2,Daniele Monteiro Ferreira,Monteiro
3,Daniel Ruschel,Ruschel
4,Elsie Franklin Guimarães,Guimar
5,Eric J Tepe,Tepe
6,Erika Erika Von Sohsten de Souza Medeiros,Medeiros
7,George Azevedo de Queiroz,Queiroz
8,Micheline Carvalho-Silva,Carvalho
9,Ricardo de la Merced Callejas Posada,Callejas


## Get variations of trusted identifiers

In [5]:
db.create_table(engine, TrustedIdentifier)

count_of_records_trusted_identifier = session.query(TrustedIdentifier) \
                                        .count()

if count_of_records_trusted_identifier == 0:
    list_full_name = list_identifier_trusted['full_name']
    list_searched_name = list_identifier_trusted['searched_name']
    for item in zip(list_full_name, list_searched_name):
        full_name_identifier = item[0]
        variation_of_name_identfier = item[1]

        query = session.query(DataSP.identified_by) \
            .filter(DataSP.identified_by.ilike('%{}%'.format(variation_of_name_identfier))) \
            .distinct(DataSP.identified_by) \
            .all()

        for q in query:
            print('full name identifier: %s variation founded: %s' % (full_name_identifier, q.identified_by))
            new_identifier_trusted = TrustedIdentifier(name=full_name_identifier, searched_name=variation_of_name_identfier, value_founded=q.identified_by, trusted=False)

            session.add(new_identifier_trusted)
            session.commit()

        count_of_records_trusted_identifier = session.query(TrustedIdentifier).count()
        print('count records inserted in table %s: %d' % (TrustedIdentifier.__tablename__, count_of_records_trusted_identifier))

create table: trusted_identifier
full name identifier: Aline Vieira de Melo Silva variation founded: A. G. da Silva
full name identifier: Aline Vieira de Melo Silva variation founded: A. S. L. da Silva
full name identifier: Aline Vieira de Melo Silva variation founded: A. V. de Melo Silva
full name identifier: Aline Vieira de Melo Silva variation founded: Bernardi, P.T.; Mendes, N.A.; Schinato, F.; Silva, A.L.
full name identifier: Aline Vieira de Melo Silva variation founded: Caralho Silva, M.
full name identifier: Aline Vieira de Melo Silva variation founded: Carvalho Silva, M
full name identifier: Aline Vieira de Melo Silva variation founded: Carvalho-Silva M
full name identifier: Aline Vieira de Melo Silva variation founded: Carvalho-Silva, M
full name identifier: Aline Vieira de Melo Silva variation founded: Carvalho-Silva, M.
full name identifier: Aline Vieira de Melo Silva variation founded: Carvalho-Silva, M.; Amorim, P.
full name identifier: Aline Vieira de Melo Silva variatio

In [10]:
count_variations_identifier_name_marked_trusted = session.query(TrustedIdentifier.value_founded) \
    .filter(TrustedIdentifier.trusted) \
    .distinct()\
    .count()

# 187
print('count of variations founded of identifiers name marked trusted: %d' % count_variations_identifier_name_marked_trusted)

count of variations founded of identifiers name marked trusted: 187


## Get list of records with variations of name identifier

In [12]:
query = session.query(TrustedIdentifier.value_founded) \
    .filter(TrustedIdentifier.trusted) \
    .distinct()\
    .all()

list_variations_of_identifiers_trusted = [q[0] for q in query]

count_of_records_with_variations_identifier_name = session.query(DataSP) \
    .filter(DataSP.identified_by.in_(list_variations_of_identifiers_trusted))\
    .count()

print('count of records founded with variations of identifier name: %d' % count_of_records_with_variations_identifier_name)

count of records founded with variations of identifier name: 13182


## Create table data by trusted identifiers

In [13]:
db.create_table(engine, DataTrustedIdentifier)

count_data_in_data_trusted_identifier = session.query(DataTrustedIdentifier).count()

if count_data_in_data_trusted_identifier == 0:
    condition = DataSP.identified_by.in_(list_variations_of_identifiers_trusted)
    query = session.query(DataSP) \
        .filter(condition) \
        .all()
    for q in query:
        new_data_of_identifier_trusted = create_data_trusted_identifier(q)
        session.add(new_data_of_identifier_trusted)

count_data_from_trusted_identifers = session.query(DataTrustedIdentifier).count()
print('count of records in table %s: %d' % (DataTrustedIdentifier.__tablename__, count_data_from_trusted_identifers))

create table: data_trusted_identifier
count of records in table data_trusted_identifier: 13182


## Close connection

In [9]:
session.close()
engine.dispose()